# Initialize libraries
reference - [DeepLearningAI-Build an agent from scratch](https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/c1l2c/build-an-agent-from-scratch)

In [1]:
import pandas as pd
import openai

In [4]:
import os
from openai import AzureOpenAI

endpoint = "https://jpati-map2omv6-eastus2.openai.azure.com/"
model_name = "gpt-4.1"
deployment = "gpt-4.1"

subscription_key = "3nGeXEgj0OJB5VMqfr7kTNhJiW0zxlT7Y14joTwfGHb7VDw18FDcJQQJ99BEACHYHv6XJ3w3AAAAACOGE5Y0"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Hyderabad, what should I see?",
        }
    ],
    max_completion_tokens=800,
    temperature=1.0,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    model=deployment
)

print(response.choices[0].message.content)

That’s exciting! Hyderabad has a fascinating blend of history, culture, cuisine, and modern attractions. Here are **must-see sights and experiences** in Hyderabad:

### Historical & Architectural Landmarks
1. **Charminar**  
   The iconic symbol of Hyderabad, this 16th-century monument is a must-visit for its stunning architecture and surrounding bustling bazaars.

2. **Golconda Fort**  
   A majestic citadel with impressive acoustics, tunnels, and viewpoints over the city—a great place for sunset.

3. **Qutb Shahi Tombs**  
   These beautifully domed tombs set in landscaped gardens are a haunting reminder of Hyderabad’s royal past.

4. **Chowmahalla Palace**  
   The opulent former seat of the Nizams, beautifully restored and open to the public.

### Culture & Museums
5. **Salar Jung Museum**  
   One of India’s largest museums, housing an eclectic collection of art, antiques, and rare artifacts from around the world.

6. **Shilparamam**  
   An arts and crafts village and cultural ce

# Define agent

## Agent
- An agent is an autonomous entity that can reason, plan, and decide which actions (tools) to use to accomplish a task.
- It manages the conversation, maintains context, and determines when and how to use tools.
Example: Your Agent class receives a user query, decides if it needs to call a tool (like a calculator or a dog weight lookup), and manages the flow of 

**Thought → Action → Observation → Answer.**

![alt text](image.png)

In [11]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [10]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model=model_name, 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

In [12]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [46]:
abot = Agent(prompt)

In [47]:
abot("how are you doing today?")

"Answer: Thank you for asking! As an AI, I don't have feelings, but I'm here and ready to help you with any questions you have."

In [48]:
abot("Do you knwo different weights of dogs?")

"Answer: Yes, I can provide information about the average weights of different dog breeds. If you tell me the breed you're interested in, I can look up its average weight for you."

In [49]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the average weight of a Toy Poodle using the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE


In [15]:
result = average_dog_weight("Toy Poodle")

In [16]:
result

'a toy poodles average weight is 7 lbs'

In [17]:
next_prompt = "Observation: {}".format(result)

In [18]:
next_prompt

'Observation: a toy poodles average weight is 7 lbs'

In [19]:
abot(next_prompt)

"Answer: A toy poodle's average weight is 7 lbs."

In [20]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [34]:
abot = Agent(prompt)

In [35]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then add them together.\nAction: average_dog_weight: Border Collie\nPAUSE'

In [36]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [37]:
abot(next_prompt)

'Action: average_dog_weight: Scottish Terrier\nPAUSE'

In [45]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user',
  'content': 'I have 2 dogs,

In [40]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [41]:
abot(next_prompt)

'Action: calculate: 37 + 20\nPAUSE'

In [28]:
calculate('37 + 20')

57

In [43]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)


Observation: 57


In [44]:
abot(next_prompt)

'Answer: The combined average weight of a Border Collie and a Scottish Terrier is 57 lbs.'